# 🧠 Entraînement de modèles de régression

Ce notebook a pour but d’entraîner différents **modèles de régression supervisée** afin de prédire le temps de résolution des tickets d’incidents.

Les étapes suivantes sont couvertes :

- Chargement du jeu de données prétraité
- Séparation en ensembles d'entraînement et de test
- Construction d’un pipeline avec transformation des variables
- Entraînement de plusieurs modèles (régression linéaire, régularisée, ensamble, etc.)
- Évaluation des performances à l’aide de métriques appropriées
- Sélection du meilleur modèle
- Sauvegarde du pipeline pour une utilisation future (API, application, etc.)


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import root_mean_squared_error,r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import time
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))


##### Chargement et exploration des données

> Chager le jeu de données nettoyé et examine ses caractéristiques principales pour comprendre la structure et la distribution des données.

In [24]:
df = pd.read_csv(os.path.join(PROJECT_ROOT, "data", "processed", "clean_trouble_tickets.csv"))
df.head()

,ticketId,category,creationDate,device,issueType,priority,severity,status,resolutionDate,customer,resolution_time,creation_day_of_week,creation_month,is_weekend
0,T4347360,9,2025-01-01 00:10:00,Gateway-CHI-V6500-WH892,267,0,2,Closed,2025-01-01 09:00:00,9905095026,8.833333,2,1,0
1,T4347363,9,2025-01-01 00:20:00,Modem-NYC-S8500-YE944,0,0,2,Closed,2025-01-01 05:52:28,9905095026,5.541111,2,1,0
2,T4347366,9,2025-01-01 00:25:00,Gateway-CHI-V6500-WH892,82,0,2,Closed,2025-01-01 09:00:00,9905095026,8.583333,2,1,0
3,T4347375,9,2025-01-01 00:40:00,Gateway-CHI-V6500-WH892,46,0,2,Closed,2025-01-01 09:00:00,9905095026,8.333333,2,1,0
4,T4347376,9,2025-01-01 00:40:00,Hub-SEA-M3000-SJ762,31,0,2,Closed,2025-01-01 09:00:00,9902641725,8.333333,2,1,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4737 entries, 0 to 4736
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ticketId              4737 non-null   object 
 1   category              4737 non-null   int64  
 2   creationDate          4737 non-null   object 
 3   device                4737 non-null   object 
 4   issueType             4737 non-null   int64  
 5   priority              4737 non-null   int64  
 6   severity              4737 non-null   int64  
 7   status                4737 non-null   object 
 8   resolutionDate        4737 non-null   object 
 9   customer              4737 non-null   int64  
 10  resolution_time       4737 non-null   float64
 11  creation_day_of_week  4737 non-null   int64  
 12  creation_month        4737 non-null   int64  
 13  is_weekend            4737 non-null   int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 518.2+ KB


##### Analyse des corrélations

> Analyse des corrélations entre les variables numériques et la variable cible (temps de résolution) pour identifier les features les plus importantes.


In [26]:
# Corrélations avec la variable cible
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlations = df[numeric_cols].corr()['resolution_time'].sort_values(ascending=False)
print("\nCorrélations avec le temps de résolution:")
print(correlations)


Corrélations avec le temps de résolution:
resolution_time         1.000000
priority                0.768150
creation_month          0.333608
is_weekend              0.147683
creation_day_of_week    0.142696
issueType               0.111931
customer                0.049525
severity                0.030258
category               -0.365367
Name: resolution_time, dtype: float64


##### Sélection des variables prédicteurs

> Définition des variables prédicteurs à utiliser dans les modèles, basées sur l'analyse préliminaire et la compréhension du domaine.



In [27]:
selected_features = [
    'priority',
    'category',
    'creation_month',
    'creation_day_of_week',
    'is_weekend',
    'issueType'
]

##### Préparation des données d'entraînement

> Division du jeu de données en ensembles d'entraînement et de test pour permettre l'évaluation objective des performances des modèles.

In [28]:
# Sélection des features à utiliser
X = df[selected_features]
y = df['resolution_time']

# Entraînement et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\nTaille de l'ensemble d'entraînement: {X_train.shape}")
print(f"Taille de l'ensemble de test: {X_test.shape}")


Taille de l'ensemble d'entraînement: (3315, 6)
Taille de l'ensemble de test: (1422, 6)


##### Définition de la fonction d'évaluation

> Création d'une fonction pour évaluer systématiquement les performances des modèles, incluant le calcul des métriques RMSE et R² et le temps d'exécution.


In [29]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):

    start_time = time.time()

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    total_time = time.time() - start_time

    train_rmse = root_mean_squared_error(y_train, y_train_pred)
    test_rmse = root_mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    results = {
        'Model': model_name,
        'Time (s)': round(total_time, 4),
        'RMSE (train)': round(train_rmse, 4),
        'RMSE (test)': round(test_rmse, 4),
        'R² (train)': round(train_r2, 4),
        'R² (test)': round(test_r2, 4)
    }
    return model,results


##### Configuration du préprocesseur

> Définition du pipeline de prétraitement pour transformer les variables catégorielles et numériques de manière cohérente.



In [30]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['creation_month', 'creation_day_of_week', 'is_weekend']),
        ('priority', OneHotEncoder(handle_unknown='ignore'), ['priority'])
    ],
    remainder='passthrough'
)


### Entraînement des modèles

Cette section comprend l'entraînement de différents modèles de régression pour prédire le temps de résolution:


In [31]:
results_list = []

##### Régression linéaire

> Entraînement d'un modèle de régression linéaire standard comme référence de base.


In [32]:
linear_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

linear_model, linear_results = evaluate_model(
    linear_pipeline, X_train, X_test, y_train, y_test,
    "Régression Linéaire"
)
results_list.append(linear_results)


##### Régression Ridge

> Application de la régularisation L2 avec optimisation du paramètre


In [33]:
param_grid = {'regressor__alpha': [0.01, 0.1, 1.0, 10.0, 50.0, 100.0]}

ridge_pipeline = Pipeline([
     ('preprocessor', preprocessor),
    ('regressor', Ridge(random_state=42))
])

grid_search = GridSearchCV(
    ridge_pipeline, param_grid, cv=5,
    scoring='neg_mean_squared_error', n_jobs=-1
)

grid_search.fit(X_train, y_train)
best_alpha = grid_search.best_params_['regressor__alpha']

best_ridge = Pipeline([
     ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=best_alpha, random_state=42))
])

ridge_model, ridge_results = evaluate_model(
    best_ridge, X_train, X_test, y_train, y_test,
    f"Régression Ridge (alpha={best_alpha})"
)

results_list.append(ridge_results)

##### Régression Lasso

> Application de la régularisation L1 avec optimisation du paramètre


In [34]:
lasso_param_grid = {
    'regression__alpha': [0.01, 0.1, 1.0, 10.0, 50.0, 100.0]
}

lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regression', Lasso(random_state=42, max_iter=10000))
])

lasso_grid = GridSearchCV(
    lasso_pipeline,
    lasso_param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

lasso_grid.fit(X_train, y_train)
best_lasso_alpha = lasso_grid.best_params_['regression__alpha']

best_lasso = Pipeline([
     ('preprocessor', preprocessor),
    ('regression', Lasso(alpha=best_lasso_alpha, random_state=42, max_iter=10000))
])

lasso_model, lasso_results = evaluate_model(
    best_lasso, X_train, X_test, y_train, y_test, f"Régression Lasso (alpha={best_lasso_alpha})")

results_list.append(lasso_results)

##### Gradient Boosting Regressor

> Entraînement d'un modèle d'ensemble basé sur le boosting pour capturer les relations non-linéaires.


In [35]:
gb_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', GradientBoostingRegressor(random_state=42))
])

gb_model, gb_results = evaluate_model(
    gb_pipeline, X_train, X_test, y_train, y_test,
    "Gradient boosting regressor"
)

results_list.append(gb_results)

##### Random Forest Regressor

> Implémentation d'un modèle d'ensemble basé sur les arbres de décision pour capturer les interactions complexes.


In [36]:
rf_pipeline = Pipeline([
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

rf_model, rf_results = evaluate_model(
    rf_pipeline, X_train, X_test, y_train, y_test,
    "Random Forest regressor"
)
results_list.append(rf_results)

##### Comparaison des modèles

> Analyse comparative des performances de tous les modèles entraînés pour sélectionner le plus approprié selon les métriques définies.


In [37]:
results_df = pd.DataFrame(results_list)

print("Comparaison des performances des modèles:")
display(results_df.sort_values(by='R² (test)', ascending=False))


Comparaison des performances des modèles:


,Model,Time (s),RMSE (train),RMSE (test),R² (train),R² (test)
3,Gradient boosting regressor,0.2397,6.3172,6.5129,0.7753,0.7653
4,Random Forest regressor,0.6896,4.2346,6.8943,0.8990,0.7371
2,Régression Lasso (alpha=0.01),0.0564,8.1830,8.1659,0.6229,0.6311
0,Régression Linéaire,0.0243,8.1823,8.1698,0.6230,0.6308
1,Régression Ridge (alpha=0.01),0.0178,8.1823,8.1698,0.6230,0.6308


##### Sauvegarde du modèle

> Exportation du pipeline complet incluant prétraitement et modèle pour une utilisation future dans une application.


In [38]:
joblib.dump(gb_pipeline, os.path.join(PROJECT_ROOT, "models", "model.pkl"))
print("Pipeline guardado como model.pkl")

Pipeline guardado como model.pkl


## ✅ Conclusion de l'entraînement

- Plusieurs modèles ont été testés : régression linéaire, Ridge, Lasso, Random Forest, Gradient Boosting, etc.
- Les performances ont été évaluées avec RMSE et R² sur les ensembles d'entraînement et de test
- Le meilleur compromis performance/généralisation a été sélectionné pour un usage ultérieur
- Le pipeline complet a été sauvegardé pour intégration dans une API

➡️ L'étape suivante consistera à **intégrer ce modèle dans une application Flask** pour effectuer des prédictions à partir de nouveaux tickets.
